In [11]:
import sys
import subprocess
import pkg_resources

required = {'numpy', 'pandas', 'tensorflow', 'keras'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    python = sys.executable
    subprocess.check_call([python, '-m', 'pip', 'install', *missing], stdout=subprocess.DEVNULL)

In [12]:
import tensorflow as tf
import numpy as np
import pandas as pd
import subprocess

## Import dataset

https://cseweb.ucsd.edu/~jmcauley/datasets.html#clothing_fit

Decomposing fit semantics for product size recommendation in metric spaces <br />
Rishabh Misra, Mengting Wan, Julian McAuley <br />
RecSys, 2018 <br />

### - download data

In [13]:
DATASET_URL = 'http://deepx.ucsd.edu/public/jmcauley/renttherunway/renttherunway_final_data.json.gz'
DATASET_PATH = './renttherunway_final_data.json'

# download and extract the data
subprocess.run(['wget', DATASET_URL])
subprocess.run(['gzip', '-d', f'{DATASET_PATH}.gz'])

# read the dataframe
dataset = pd.read_json(DATASET_PATH, lines=True)
dataset

,fit,user_id,bust size,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,review_date
0,fit,420272,34d,2260466,137lbs,10.0,vacation,An adorable romper! Belt and zipper were a lit...,hourglass,So many compliments!,romper,"5' 8""",14,28.0,"April 20, 2016"
1,fit,273551,34b,153475,132lbs,10.0,other,I rented this dress for a photo shoot. The the...,straight & narrow,I felt so glamourous!!!,gown,"5' 6""",12,36.0,"June 18, 2013"
2,fit,360448,NaN,1063761,NaN,10.0,party,This hugged in all the right places! It was a ...,NaN,It was a great time to celebrate the (almost) ...,sheath,"5' 4""",4,116.0,"December 14, 2015"
3,fit,909926,34c,126335,135lbs,8.0,formal affair,I rented this for my company's black tie award...,pear,Dress arrived on time and in perfect condition.,dress,"5' 5""",8,34.0,"February 12, 2014"
4,fit,151944,34b,616682,145lbs,10.0,wedding,I have always been petite in my upper body and...,athletic,Was in love with this dress !!!,gown,"5' 9""",12,27.0,"September 26, 2016"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192539,fit,66386,34dd,2252812,140lbs,10.0,work,Fit like a glove!,hourglass,LOVE IT!!! First Item Im thinking of buying!,jumpsuit,"5' 9""",8,42.0,"May 18, 2016"
192540,fit,118398,32c,682043,100lbs,10.0,work,The pattern contrast on this dress is really s...,petite,LOVE it!,dress,"5' 1""",4,29.0,"September 30, 2016"
192541,fit,47002,36a,683251,135lbs,6.0,everyday,"Like the other DVF wraps, the fit on this is f...",straight & narrow,"Loud patterning, flattering fit",dress,"5' 8""",8,31.0,"March 4, 2016"
192542,fit,961120,36c,126335,165lbs,10.0,wedding,This dress was PERFECTION. it looked incredib...,pear,loved this dress it was comfortable and photog...,dress,"5' 6""",16,31.0,"November 25, 2015"


### - apply preprocessing 

In [14]:
# drop redundant columns
columns_to_drop = ['review_date']
dataset = dataset.drop(columns_to_drop, axis = 1)

In [15]:
# one-hot encoding
columns_to_encode = ['rented for', 'body type', 'category']
one_hot = pd.get_dummies(dataset[columns_to_encode])

# drop encoded colunns and add one-hot versions
dataset = dataset.drop(columns_to_encode, axis = 1)
dataset = dataset.join(one_hot)
dataset

,fit,user_id,bust size,item_id,weight,rating,review_text,review_summary,height,size,age,rented for_date,rented for_everyday,rented for_formal affair,rented for_other,rented for_party,rented for_party: cocktail,rented for_vacation,rented for_wedding,rented for_work,body type_apple,body type_athletic,body type_full bust,body type_hourglass,body type_pear,body type_petite,body type_straight & narrow,category_ballgown,category_blazer,category_blouse,category_blouson,category_bomber,category_buttondown,category_caftan,category_cami,category_cape,category_cardigan,category_coat,category_combo,category_crewneck,...,category_kimono,category_knit,category_legging,category_leggings,category_maxi,category_midi,category_mini,category_overalls,category_overcoat,category_pant,category_pants,category_parka,category_peacoat,category_poncho,category_print,category_pullover,category_romper,category_sheath,category_shift,category_shirt,category_shirtdress,category_skirt,category_skirts,category_skort,category_suit,category_sweater,category_sweatershirt,category_sweatpants,category_sweatshirt,category_t-shirt,category_tank,category_tee,category_tight,category_top,category_trench,category_trouser,category_trousers,category_tunic,category_turtleneck,category_vest
0,fit,420272,34d,2260466,137lbs,10.0,An adorable romper! Belt and zipper were a lit...,So many compliments!,"5' 8""",14,28.0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,fit,273551,34b,153475,132lbs,10.0,I rented this dress for a photo shoot. The the...,I felt so glamourous!!!,"5' 6""",12,36.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,fit,360448,NaN,1063761,NaN,10.0,This hugged in all the right places! It was a ...,It was a great time to celebrate the (almost) ...,"5' 4""",4,116.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,fit,909926,34c,126335,135lbs,8.0,I rented this for my company's black tie award...,Dress arrived on time and in perfect condition.,"5' 5""",8,34.0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,fit,151944,34b,616682,145lbs,10.0,I have always been petite in my upper body and...,Was in love with this dress !!!,"5' 9""",12,27.0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192539,fit,66386,34dd,2252812,140lbs,10.0,Fit like a glove!,LOVE IT!!! First Item Im thinking of buying!,"5' 9""",8,42.0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
192540,fit,118398,32c,682043,100lbs,10.0,The pattern contrast on this dress is really s...,LOVE it!,"5' 1""",4,29.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
192541,fit,47002,36a,683251,135lbs,6.0,"Like the other DVF wraps, the fit on this is f...","Loud patterning, flattering fit","5' 8""",8,31.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
192542,fit,961120,36c,126335,165lbs,10.0,This dress was PERFECTION. it looked incredib...,loved this dress it was comfortable and photog...,"5' 6""",16,31.0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,

## Train NN  